In [17]:
import glob
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup

In [19]:
news = pd.read_csv('./NONen_interim_01_05_ALL_IN.csv', index_col=False)

/Users/seyi/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
news.head()

,Unnamed: 0,index,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,...,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL,title
0,0,43,896053216,19200101,192001,1920,1920.0027,NaN,NaN,NaN,...,"New Delhi, Delhi, India",IN,IN07,17911.0,28.6000,77.2000,-2106102,20200101001500,https://www.anandabazar.com/national/every-pol...,every police bullet is aimed at the constituti...
1,1,137,896053310,19200101,192001,1920,1920.0027,COP,POLICE,NaN,...,"Ambajogai, Maharashtra, India",IN,IN16,70179.0,18.7333,76.3833,-2088642,20200101001500,https://maharashtratimes.indiatimes.com/mahara...,impersonation bans a police officer
2,2,202,896053375,19200101,192001,1920,1920.0027,EDU,STUDENT,NaN,...,"Mumbai, Maharashtra, India",IN,IN16,70184.0,18.9750,72.8258,-2092174,20200101001500,https://maharashtratimes.indiatimes.com/mahara...,welcome to the new year saying write date
3,3,212,896053385,19200101,192001,1920,1920.0027,FRA,FRANCE,FRA,...,"Calcutta, West Bengal, India",IN,IN28,70294.0,22.5697,88.3697,-2092511,20200101001500,https://www.anandabazar.com/editorial/end-of-p...,end of participatory democracy right wing popu...
4,4,223,896053396,19200101,192001,1920,1920.0027,GOV,PRESIDENT,NaN,...,"Pune, Maharashtra, India",IN,IN16,70178.0,18.5333,73.8667,-2108361,20200101001500,https://maharashtratimes.indiatimes.com/mahara...,bjps retreat due to their words


In [21]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
def get_full_text(url):
    try:
        req = Request(url,headers=header)
        news_article = BeautifulSoup(urlopen(req), features="html.parser").get_text() #non-english
#         news_article = BeautifulSoup(urlopen(req), features="html.parser", from_encoding="iso-8859-1").get_text() #english
    except:
        news_article = 'N/A'
    return news_article

In [22]:
def process_data(news_subset):
    news_subset["text"] = news_subset["SOURCEURL"].apply(get_full_text)
    news_subset["text"]  = news_subset["text"].apply(lambda x: str(x).encode("utf-8", errors="ignore").decode("utf-8", errors="ignore")) #ignore surrogate character errors
    news_subset.to_csv("./NONen_interim_01_05_ALL_IN_with_full_texts.csv",
                       mode='a', header=False)#output file already created with header

In [ ]:
# split up into data chunks for large dataset
split_size = int(news.shape[0] / 1000)
for start in range(0, news.shape[0], split_size):
    news_subset = news.iloc[start:start + split_size]
    process_data(news_subset)

In [23]:
news_df = pd.read_csv("./NONen_interim_01_05_ALL_IN_with_full_texts.csv", encoding='utf-8')

/Users/seyi/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [24]:
news_df.columns

Index(['Unnamed: 0', 'index', 'GLOBALEVENTID', 'SQLDATE', 'MonthYear', 'Year',
       'FractionDate', 'Actor1Code', 'Actor1Name', 'Actor1CountryCode',
       'Actor1KnownGroupCode', 'Actor1EthnicCode', 'Actor1Religion1Code',
       'Actor1Religion2Code', 'Actor1Type1Code', 'Actor1Type2Code',
       'Actor1Type3Code', 'Actor2Code', 'Actor2Name', 'Actor2CountryCode',
       'Actor2KnownGroupCode', 'Actor2EthnicCode', 'Actor2Religion1Code',
       'Actor2Religion2Code', 'Actor2Type1Code', 'Actor2Type2Code',
       'Actor2Type3Code', 'IsRootEvent', 'EventCode', 'CAMEOCodeDescription',
       'EventBaseCode', 'EventRootCode', 'QuadClass', 'GoldsteinScale',
       'NumMentions', 'NumSources', 'NumArticles', 'AvgTone', 'Actor1Geo_Type',
       'Actor1Geo_FullName', 'Actor1Geo_CountryCode', 'Actor1Geo_ADM1Code',
       'Actor1Geo_ADM2Code', 'Actor1Geo_Lat', 'Actor1Geo_Long',
       'Actor1Geo_FeatureID', 'Actor2Geo_Type', 'Actor2Geo_FullName',
       'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Cod

In [25]:
news_df[news_df['text'] == 'N/A'].count()

Unnamed: 0             0
index                  0
GLOBALEVENTID          0
SQLDATE                0
MonthYear              0
                      ..
ActionGeo_FeatureID    0
DATEADDED              0
SOURCEURL              0
title                  0
text                   0
Length: 66, dtype: int64